In [1]:
# Pluto imports
import Pluto
from Pluto import GLM as glm
from Pluto import GLFW as glfw
from Pluto import EventSystem
from Pluto import RenderSystem
from Pluto import Prefabs

import HelperClasses
from HelperClasses import Model
from HelperClasses import Camera
from HelperClasses import Light

import threading
import time
import math
import colorsys
import numpy as np
import scipy
import scipy.linalg

In [2]:
# Initialize Pluto
Pluto.Initialize(True, False, []) # 

In [3]:
# Get System/Library Handles
es = EventSystem.Get()
rs = RenderSystem.Get()
g = glfw.Get()

quality = 1.
cam = Prefabs.CreatePrefabCamera(
   mode="arcball", #can also be arcball 
   width = 1920//4, 
   height = 1080//4, 
   fov = 45, 
   msaa_samples = 1, 
   target = 10, 
   enable_depth_prepass = False)
cam.transform.set_position(0.0, -5.0, 0.0)

In [4]:
es.resize_window("prefab_window", 640, 360)

In [5]:
# Make a dome light
rs.set_top_sky_color(glm.vec3(.01, .01, .01))
rs.set_bottom_sky_color(glm.vec3(.01, .01, .01))
rs.set_sky_transition(20.)
rs.set_gamma(2.2)
rs.set_exposure(1.0)

In [6]:
# Make some lights
MainLight = Light(
    Pluto.Entity.Create("MainLight"), 
    Pluto.Mesh.CreateBox("MainLight"), 
    Pluto.Transform.Create("MainLight"), 
    Pluto.Material.Create("MainLight"), 
    Pluto.Light.Create("MainLight"))

In [7]:
MainLight.light.use_plane()
MainLight.transform.set_position(0, 0, 3.0)
MainLight.transform.set_scale(1., 1., .01)
MainLight.light.set_temperature(4500)
MainLight.light.set_intensity(10.0)
MainLight.light.cast_shadows(True)
MainLight.light.set_double_sided(True)
MainLight.transform.set_rotation(0, glm.vec3(1.0, 0.0, 0.0))

In [8]:
# Make floor
floor = Model(Pluto.Entity.Create(str(0)), Pluto.Mesh.CreatePlane("Floor"), Pluto.Transform.Create(str(0)), Pluto.Material.Create(str(0)))
floor.transform.set_position(0.0, 0.0, -1.0)
floor.transform.set_scale(1000.0, 1000.0, 1000.0)
floor.material.set_base_color(1, 1, 1, 1.0)
floor.material.set_roughness(.1) 
floor.material.set_metallic(.0)
# floor.material.show_position()

In [9]:
# Make Cloth
size = 10
positions = [None for _ in range(size*size)]
normals = [None for _ in range(size*size)]
colors = [None for _ in range(size*size)]
texcoords = [None for _ in range(size*size)]
indices = []
edges = []
rest_lengths = []

for x in range(0, size):
    for y in range(0, size):
            positions[x*size + y] = (glm.vec3(x, y, 0) * 1.0 / size)
            normals[x*size + y] = (glm.vec3(0,0,1))
            texcoords[x*size + y] = (glm.vec2(x, y)* 1.0 / size)
            colors[x*size + y] = (glm.vec4(1,1,1,1))

off = 0
for x in range(0, size-1):
    for y in range(0, size-1):
        i1 = (x) + (y) * size
        i2 = (x + 1) + (y + 1) * size
        i3 = (x) + (y + 1) * size
        i4 = (x) + (y) * size
        i5 = (x + 1) + (y) * size
        i6 = (x + 1) + (y + 1) * size
        indices.extend([i1,i3,i2,i4,i6,i5])
        
# Make an entity with a mesh we can edit        
custom_mesh = Pluto.Mesh.CreateFromRaw("grid", positions, normals, colors, texcoords, indices, edges, rest_lengths, True)
custom_mesh.compute_simulation_matrices()
custom_mesh.set_damping_factor(0.00001)
model = Model(Pluto.Entity.Create("custom_mesh"), custom_mesh, Pluto.Transform.Create("custom_mesh"), Pluto.Material.Create("custom_mesh"))
model.transform.set_scale(2.0, 2.0, 2.0)
model.material.set_roughness(0.3)
model.material.set_metallic(0.0)
model.material.set_base_color(1, 0, 0, 1)
model.transform.set_position(-1.0, -1.0, 0.0)
#model.material.show_normals()

In [10]:
# Make spring constraints
f_ext = glm.vec3(0.0,0.0,-9.8)

mesh_height1 = .5
mesh_height2 = .5
mesh_height3 = .5
mesh_height4 = .5
stretch = 1.0


In [11]:
# Start interactive loop
last_time = time.time()

print("Press escape to quit")
print("Press alt+enter to go fullscreen")
def loop () :
    global stop
    global iteration
    stop = False
    
    global mesh_height1
    global mesh_height2
    global mesh_height3
    global mesh_height4

    last_time = time.time()
    iterations = 0

    while g.get_key_action("prefab_window", glfw.get_key_code("ESCAPE")) == 0 and stop == False:
        start_time = time.time()
        if start_time - last_time > .01:
            last_time = start_time

            mesh_height1 = .0 + .1 * math.sin(last_time)
            mesh_height2 = .1 + .1 * math.sin(last_time * 2.0)
            mesh_height3 = .5 + .1 * math.sin(last_time * 3.0)
            mesh_height4 = .6 + .1 * math.sin(last_time * 4.0)

            custom_mesh.update()

            # set anchors
            custom_mesh.edit_position(0, glm.vec3(0.0, 0.0, mesh_height1))
            custom_mesh.edit_position((size-1)*size, glm.vec3(stretch, 0.0, mesh_height2))
            custom_mesh.edit_position(size-1, glm.vec3(0.0, stretch, mesh_height3))
            custom_mesh.edit_position((size-1)*(size+1), glm.vec3(stretch, stretch, mesh_height4))
            custom_mesh.edit_velocity(0, glm.vec3(0.0,0.0,0.0))
            custom_mesh.edit_velocity((size-1)*size, glm.vec3(0.0,0.0,0.0))
            custom_mesh.edit_velocity(size-1, glm.vec3(0.0,0.0,0.0))
            custom_mesh.edit_velocity((size-1)*(size+1), glm.vec3(0.0,0.0,0.0))
            
            custom_mesh.compute_smooth_normals()
        else :
            time.sleep(.01)

try:
    stop = True
    t.join()
except NameError:
    pass

t = threading.Thread(target=loop)
t.start()

Press escape to quit
Press alt+enter to go fullscreen


In [16]:
# we can set stiffness, mass and damping factor dynamically 
#custom_mesh.set_stiffness(100)
#custom_mesh.set_mass(10)
#custom_mesh.set_damping_factor(0.001)